In [1]:
from datetime import datetime
print(f'Päivitetty {datetime.now().date()} / Aki Taanila')

Päivitetty 2025-10-19 / Aki Taanila


# Annuiteettilainojen havainnollistaminen ja vertailu

Tässä muistiossa tarkastellaan ja vertaillaan kuukausittain lyhennettäviä annuitettilainoja. Tulokset viedään Exceliin **xlwings**-paketin toimintojen avulla.

In [2]:
import pandas as pd
from datetime import date
import xlwings as xw
import annuiteetti  #oletus: annuiteetti.py-moduuli on tallennettu samaan kansioon tämän muistion kanssa

## Lähtötiedot

Täydennä seuraavaan soluun lainan lähtötiedot:
- korkokanta desimaalimuodossa; esimerkiksi ,0245 tarkoittaa 2,45 % korkokantaa.
- aika_kuukausina: aseta laina-ajaksi 0, jos haluat laskea lainan pituuden maksuerän perusteella.
- maksuerä: aseta maksueräksi 0, jos haluat laskea lainan maksuerän lainan pituuden perusteella.

In [3]:
aloituspäivä = date(2026, 1, 1)
laina = 300000
korkokanta = .0245
aika_kuukausina = 0
maksuerä = 1500

## Lyhennystaulukon luonti

Tässä luodaan lyhennystaulukko **annuiteetti.py**-moduulin **lyhennystaulukko**-funktiolla.

In [4]:
taulukko, tiedot = annuiteetti.lyhennystaulukko(aloituspäivä, laina, korkokanta, aika_kuukausina, maksuerä)

## Vertailun lähtötiedot

- aloituspäivä on sama kaikille vertailtaville lainoille.
- lainat, korkokannat, ajat_kuukausina, maksuerät: kirjoita jokaiseen yhtä monta vaihtoehtoa. Vaihtoehtoja voit kirjoittaa enemmänkin kuin kolme.
- jos ajat_kuukausina=0, niin vastaavan maksuerän pitää olla nollaa suurempi.
- jos maksuerät=0, niin vastaavan ajat_kuukausina pitää olla nollaa suurempi.

In [5]:
aloituspäivä = date(2026, 1, 1)
lainat = (300000, 300000, 300000)
korkokannat = (.0245, .0245, .0245)
ajat_kuukausina = (0, 0, 0)
maksuerät = (1300, 1500, 1700)

## Vertailu-taulukon luonti

Tässä luodaan vertailu-taulukko **annuiteetti.py**-moduulin **lyhennystaulukko**-funktion avulla.

In [6]:
vertailu = pd.DataFrame(columns = ['Lainan suuruus', 'Korkokanta', 'Aika kuukausina', 'Maksuerä', 
                                   'Korot yhteensä', 'Viimeinen maksuerä'])
for i in range(len(lainat)): # len(lainat) antaa vertailujen vaihtoehtojen lukumäärän 
    v_taulukko, v_tiedot = annuiteetti.lyhennystaulukko(aloituspäivä=aloituspäivä, laina=lainat[i], korkokanta=korkokannat[i], 
                                            aika_kuukausina=ajat_kuukausina[i], maksuerä=maksuerät[i])
    vertailu.loc[i] = v_tiedot.T

## Lyhennystaulukko ja vertailu-taulukko Exceliin

Seuraavassa luodaan Excel-työkirja, viedään tiedot työkirjaan ja tehdään joitain muotoiluja. Apuna käytetään **xlwings**-pakettia.

Lisätietoa https://docs.xlwings.org/en/latest/

In [7]:
# Piilotettu Excel-instanssi
# Piilotuksen ansiosta käyttäjä ei pääse sotkemaan työkirjaa kesken kirjoittamisen
app = xw.App(visible=False)

# Luodaan työkirja, johon viitataan wb-nimellä
wb = app.books[0]

# Taulukkovälilehdet Exceliin
ws1= wb.sheets.active
ws1.name = 'lyhennystaulukko'
ws2 = wb.sheets.add('vertailu', after=wb.sheets.count)

# Lyhennystaulukko Exceliin
rivi = 3
sarake = 1
ws1.range(1, sarake).value = 'Annuiteettilainan lyhennystaulukko'
ws1.range(1, sarake).font.bold = True
ws1.range(rivi, sarake).value = tiedot
ws1.range((rivi+tiedot.shape[0]+1, sarake)).options(index=False).value = taulukko
ws1.used_range.autofit()

# Vertailu Exceliin
ws2.range(1, sarake).value = 'Vaihtoehtojen vertailu'
ws2.range(1, sarake).font.bold = True
ws2.range(rivi, sarake).options(index=False).value = vertailu
ws2.used_range.autofit()

ws1.activate()

# Excel-instanssi näkyville
app.visible = True
wb.activate(True)